<a href="https://colab.research.google.com/github/kkrusere/youTube-comments-Analyzer/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [784 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,920 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu j

In [3]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
import chromedriver_autoinstaller

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()
driver = webdriver.Chrome(options=chrome_options)


In [4]:
from selenium.webdriver.common.by import By

In [5]:
from google.colab import userdata
api_key = userdata.get('YouTubeAPI_key')

In [8]:
import googleapiclient.discovery
import datetime

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# YouTube video URL
video_url = "https://www.youtube.com/watch?v=cZlsZwcIgpc"

# Extract video ID from URL
video_id = video_url.split('=')[-1]

# Create YouTube API client
youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)

# Request video details
response = youtube.videos().list(
    part="snippet,statistics",
    id=video_id
).execute()

# Extract required information
video = response["items"][0]
title = video["snippet"]["title"]
thumbnail_url = video["snippet"]["thumbnails"]["default"]["url"]
view_count = video["statistics"]["viewCount"]
like_count = video["statistics"]["likeCount"]
date_posted = video["snippet"]["publishedAt"]
description = video["snippet"]["description"]


# Split the description by newlines
paragraphs = description.split("\n")

# Get the first paragraph
first_paragraph = paragraphs[0]

# Generate HTML to display the thumbnail
thumbnail_html = f'<img src="{thumbnail_url}" alt="Thumbnail">'

# Print the information
print("Title:", title)
print("Thumbnail URL:", thumbnail_html)
print("View count:", view_count)
print("Like count:", like_count)
print("Date posted:", date_posted)
print("Description:", first_paragraph)


Title: Why EVs Are Piling Up At Dealerships In The U.S.
Thumbnail URL: <img src="https://i.ytimg.com/vi/cZlsZwcIgpc/default.jpg" alt="Thumbnail">
View count: 4485912
Like count: 36460
Date posted: 2023-10-16T16:26:47Z
Description: In August 2023, it took about twice as long to sell an EV in the U.S. as it did the previous January. Prices of EVs are down 22% year-over-year and that's mainly driven by Tesla. About two thirds of EVs sold are Elon Musk's brand. Companies like Ford have ramped up hybrid production as demand has leveled off. While slightly more than half of consumers say EVs are the future and will eventually replace Internal Combustion Engines, less than a third of dealers say so. This all comes at a time when investments in EVs are more than ever. So what's really going on? Watch the video to learn more.
